In [ ]:
#require "pkp"

In [ ]:
open Owl
open Gp
open Pkp.Neuron

In [ ]:
let _ = Pkp.Misc.quiet_owl ()

In [ ]:
let plot_response (t, h, r) =
  let xr = Mat.min' t, Mat.max' t in
  let fig (module P : Plot) =
    P.plot
      (L [ t; r ])
      [ barebone
      ; margins [ `left 0.2; `right 0.95; `top 0.95; `bottom 0.6 ]
      ; borders [ `left ]
      ; xrange xr
      ; ytics `auto
      ; ylabel "r (Hz)"
      ];
    P.plot
      (L [ t; h ])
      [ barebone
      ; margins [ `left 0.2; `right 0.95; `top 0.55; `bottom 0.2 ]
      ; borders [ `left; `bottom ]
      ; xrange xr
      ; xtics `auto
      ; ytics `auto
      ; xlabel "time (s)"
      ; ylabel "h"
      ]
  in
  Juplot.draw ~size:(400, 300) fig

Simulation of a rate model:

In [ ]:
let simulate ~duration ~tau f h =
  let drdt r t = Mat.((neg r +$ f (h t)) /$ tau) in
  let open Owl_ode in
  let tspec = Types.(T1 { t0 = 0.; duration; dt = 1E-4 }) in
  let t, r =
    Ode.odeint Native.D.(rk45 ~tol:1E-5 ~dtmax:1E-3) drdt Mat.(zeros 1 1) tspec ()
  in
  let h = Mat.map h t in
  t, h, r

In [ ]:
let t, h, r =
  simulate
    ~duration:0.15
    ~tau:20E-3
    (fun x -> if x < 1. then 0. else x -. 1.)
    (fun t -> if t < 0.04 then 0. else 20.)s

In [ ]:
plot_response (t, h, r)